In [1]:
import pinocchio as pin 
import numpy as np
from numpy.linalg import pinv,inv,norm,svd,eig
from tools import collision, getcubeplacement, setcubeplacement, projecttojointlimits
from config import LEFT_HOOK, RIGHT_HOOK, LEFT_HAND, RIGHT_HAND, EPSILON
from config import CUBE_PLACEMENT, CUBE_PLACEMENT_TARGET

from tools import setcubeplacement

pybullet build time: Sep 22 2025 15:07:41


In [2]:
def find_collision_free_gradient(robot, q_init, max_iterations=50, step_size=0.05):
    """
    Find collision-free configuration using gradient-based approach.
    Moves away from collision by checking collision distances.
    
    Parameters:
    -----------
    robot : robot model with collision checking
    q_init : initial configuration (possibly in collision)
    max_iterations : maximum number of gradient steps
    step_size : step size for gradient descent
    
    Returns:
    --------
    q_free : collision-free configuration
    success : boolean indicating if a solution was found
    """
    
    def coll(q):
        """Check if configuration is in collision"""
        pin.updateGeometryPlacements(robot.model, robot.data, 
                                     robot.collision_model, robot.collision_data, q)
        return pin.computeCollisions(robot.collision_model, robot.collision_data, False)
    
    if not coll(q_init):
        return q_init.copy(), True
    
    q = q_init.copy()
    
    for i in range(max_iterations):
        # Compute numerical gradient
        gradient = np.zeros(len(q))
        
        for j in range(len(q)):
            q_plus = q.copy()
            q_minus = q.copy()
            
            delta = 0.01
            q_plus[j] += delta
            q_minus[j] -= delta
            
            # Simple collision score (0 if free, 1 if collision)
            score_plus = 1.0 if coll(q_plus) else 0.0
            score_minus = 1.0 if coll(q_minus) else 0.0
            
            gradient[j] = (score_plus - score_minus) / (2 * delta)
        
        # Move away from collision
        if np.linalg.norm(gradient) > 1e-6:
            q = q - step_size * gradient / np.linalg.norm(gradient)
        else:
            # Add random perturbation if stuck
            q = q + np.random.randn(len(q)) * 0.02
        
        # Project to joint limits
        q = np.clip(q, robot.model.lowerPositionLimit, robot.model.upperPositionLimit)
        
        if not coll(q):
            print(f"Found collision-free config after {i+1} gradient steps")
            print(f"Max joint deviation: {np.max(np.abs(q - q_init)):.4f} rad")
            return q, True
    
    print(f"Could not find collision-free config after {max_iterations} gradient steps")
    return q, False

In [3]:
def computeqgrasppose(robot, qcurrent, cube, cubetarget, viz=None):
    '''Return a collision free configuration grasping a cube at a specific location and a success flag'''
    setcubeplacement(robot, cube, cubetarget)
    q , success = find_collision_free_gradient(robot, qcurrent, 1000)
    print(q)
    viz.display(q)
    return q, False

In [4]:
def distance(q1,q2):    
    '''Return the euclidian distance between two configurations'''
    return np.linalg.norm(q2-q1)
        
def NEAREST_VERTEX(G,q_rand):
    '''returns the index of the Node of G with the configuration closest to q_rand  '''
    min_dist = 10e4
    idx=-1
    for (i,node) in enumerate(G):
        dist = distance(node[1],q_rand) 
        if dist < min_dist:
            min_dist = dist
            idx = i
    return idx

In [5]:
def ADD_EDGE_AND_VERTEX(G,parent,q):
    G += [(parent,q)]

In [6]:
# %load tp4/generated/rrt_new_conf_valid_edge
def lerp(q0,q1,t):    
    return q0 * (1 - t) + q1 * t

def NEW_CONF(q_near,q_rand,discretisationsteps, delta_q = None):
    '''Return the closest configuration q_new such that the path q_near => q_new is the longest
    along the linear interpolation (q_near,q_rand) that is collision free and of length <  delta_q'''
    q_end = q_rand.copy()
    dist = distance(q_near, q_rand)
    if delta_q is not None and dist > delta_q:
        #compute the configuration that corresponds to a path of length delta_q
        q_end = lerp(q_near,q_rand,delta_q/dist)
        # now dist == delta_q
    dt = 1 / discretisationsteps
    for i in range(1,discretisationsteps):
        q = lerp(q_near,q_end,dt*i)
        if coll(q):
            return lerp(q_near,q_end,dt*(i-1))
    return q_end


def VALID_EDGE(q_new,q_goal,discretisationsteps):
    return np.linalg.norm(q_goal -NEW_CONF(q_new, q_goal,discretisationsteps)) < 1e-3


In [7]:
# %load tp0/generated/simple_path_planning_coll
def coll(q):
     '''Return true if q in collision, false otherwise.'''
     pin.updateGeometryPlacements(robot.model,robot.data,robot.collision_model,robot.collision_data,q)
     return pin.computeCollisions(robot.collision_model,robot.collision_data,False)

def RAND_CONF(robot, checkcollision=True):
    '''
    Return a random configuration, not is collision
    '''
    while True:
        q = pin.randomConfiguration(robot.model)  # sample between -3.2 and +3.2.
        if not (checkcollision and coll(q)): return q
        

In [11]:
discretisationsteps_newconf = 200 #To tweak later on
discretisationsteps_validedge = 200 #To tweak later on
k = 1000  #To tweak later on
delta_q = 3. #To tweak later on

In [12]:
def rrt(q_init, q_goal, k, delta_q, robot):
    G = [(None,q_init)]
    for _ in range(k):
        q_rand = RAND_CONF(robot)   
        q_near_index = NEAREST_VERTEX(G,q_rand)
        q_near = G[q_near_index][1]        
        q_new = NEW_CONF(q_near,q_rand,discretisationsteps_newconf, delta_q = None)    
        ADD_EDGE_AND_VERTEX(G,q_near_index,q_new)
        if VALID_EDGE(q_new,q_goal,discretisationsteps_validedge):
            print ("Path found!")
            ADD_EDGE_AND_VERTEX(G,len(G)-1,q_goal)
            return G, True
    print("path not found")
    return G, False
            
       


In [13]:
def getpath(G):
    path = []
    node = G[-1]
    while node[0] is not None:
        path = [node[1]] + path
        node = G[node[0]]
    path = [G[0][1]] + path
    return path

from math import ceil
from time import sleep

def displayedge(q0,q1,vel=2.): #vel in sec.    
    '''Display the path obtained by linear interpolation of q0 to q1 at constant velocity vel'''
    dist = distance(q0,q1)
    duration = dist / vel    
    nframes = ceil(48. * duration)
    f = 1./48.
    for i in range(nframes-1):
        viz.display(lerp(q0,q1,float(i)/nframes))
        sleep(f)
    viz.display(q1)
    sleep(f)
    
def displaypath(path):
    for q0, q1 in zip(path[:-1],path[1:]):
        displayedge(q0,q1)

In [15]:
if __name__ == "__main__":
    from tools import setupwithmeshcat
    from setup_meshcat import updatevisuals
    robot, cube, viz = setupwithmeshcat()
    
    q = robot.q0.copy()
    
    q0,successinit = computeqgrasppose(robot, q, cube, CUBE_PLACEMENT, viz)
    G, foundpath = rrt(q0, RAND_CONF(robot), k, delta_q, robot)
#     qe,successend = computeqgrasppose(robot, q, cube, CUBE_PLACEMENT_TARGET,  viz)
    path = foundpath and getpath(G) or []
    displaypath(path)
    
#     updatevisuals(viz, robot, cube, q0)

Wrapper tries to connect to server <tcp://127.0.0.1:6014>
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7014/static/
Found collision-free config after 230 gradient steps
Max joint deviation: 0.5162 rad
[-0.24051905  0.17957029 -0.02087817  0.2116478   0.04070837 -0.24179512
  0.16557577  0.15497167  0.47355967  0.05279953 -0.458129    0.16209783
 -0.51618076  0.33426491 -0.01076589]
Path found!
